# **Github project download**

In [0]:
import os.path

if os.path.isdir('./korean_word_embedding_tutorial') == 'True':  
  !rm -r ./korean_word_embedding_tutorial
  !git clone https://github.com/sseol11/korean_word_embedding_tutorial
else: 
  !git clone https://github.com/sseol11/korean_word_embedding_tutorial
    
# github에 있는 샘플 데이터 가져오는 코드

Cloning into 'korean_word_embedding_tutorial'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 41 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [0]:
# 디렉터리 파일 만드는 명령어.
!mkdir korean_word_embedding_tutorial/training_data

# **Training data download**

In [0]:
# 구글드라이브도 가능하지만, 전처리를 좀 해야한다.


#!wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1SaIMXjnsCHakT7sDo2ATaUUZb1k9x3No' -O pos_remove_wiki_ko.txt
!wget "https://www.dropbox.com/s/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt?dl=0" -O pos_remove_wiki_ko.txt
!mv pos_remove_wiki_ko.txt ./korean_word_embedding_tutorial/training_data

--2019-07-30 04:25:12--  https://www.dropbox.com/s/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt [following]
--2019-07-30 04:25:13--  https://www.dropbox.com/s/raw/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca2400c3f55e1d3308800110e18.dl.dropboxusercontent.com/cd/0/inline/AlpLOFPNmGMEw1YZfMCBf_Zc0yguQoaf9iQ6fwBmw44IiZVJOIeUNZPvJw1_pJmfsZZTNFiKbpzWFe5aLpHKwPQi_-TLcL1ShxPcH0qZyJvRVQ/file# [following]
--2019-07-30 04:25:13--  https://uca2400c3f55e1d3308800110e18.dl.dropboxusercontent.com/cd/0/inline/AlpLOFPNmGMEw1YZfMCBf_Zc0yguQoaf9iQ6fwBmw44IiZVJOIeUNZPvJw1_pJmfsZZTNFiKbpzWFe5aLp

In [0]:
# 라인이 몇개가 있는지 확인하는 것.
!wc -l ./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt

1000000 ./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt


# Environment check

In [0]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:' # 코랩에서 제공해주는 환경 정보. (Core)

Core(s) per socket:  1


In [0]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core' # 코랩에서 제공해주는 환경 정보. 

Thread(s) per core:  2


# **1. Training**

In [0]:
from gensim.models import word2vec # 젠심 사용한다.
import logging # 로깅 : 진행상황 뿌려주는 라이브러리

In [0]:
def main():
    # import modules & set up logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # read sentences, Iterate over a file that contains sentences
    # sentece를 병렬적으로 읽어온다. 파이썬 read방식이 아니라 병렬적으로 빠르게 읽어옴.
    sentences = word2vec.LineSentence('./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt')
    
    
    # sg : skip gram을 사용한다는 뜻, window : 논문 제안 저자가 5일때가 제일 좋다고 해서 ,
    # size : dimemsion size. dense vector의 사이즈 [0.3  1.2 -0.2 ... 0.9] dimension이 300개 있다.
    # 300일때가 제일 성능이 잘나와서 정했다. RNN CNN등 network에 따라서 dimension다르게 준다.
    # workers : thread 갯수 개인환경이면 코어갯수에 맞추는게 좋다.
    model = word2vec.Word2Vec(sentences,window = 5, sg = 1, size = 300, workers = 2)
    
    
    model.save('./korean_word_embedding_tutorial/wiki_ko_model')
    #model.save('./korean_word_embedding_tutorial/wiki_ko_model.cpkt') 텐서플로우 모델은 cpkt를 붙임. checkpoint
    


In [0]:
def model_example():
  
  try:
    # model학습 후에 load
    model = word2vec.Word2Vec.load('./korean_word_embedding_tutorial/wiki_ko_model')
    
    # model이 wordvector 서울이라는 단어의 wordvector를 추출 해본다. 300차원에 있는 것
    # 어느 위치에 있고 주변에 뭐가있는지 확인해봄.
    vector = model.wv['서울']
    print(vector)
  except KeyError:
    print("This word not in vocabulary")
    
  try:
    target_word = ['서울']
    # 서울이란 단어랑 가장 근접한 것이 뭔지 보여주는 것. positive한 것 상위 6개
    print(model.wv.most_similar(positive=target_word, topn=6))
  except KeyError:
    print("This word not in vocabulary")
  

In [0]:
if __name__ == "__main__":
#     main()
    model_example()

  # 각 개체와 cosine similarity를 보여준다.

2019-07-30 05:04:57,843 : INFO : loading Word2Vec object from ./korean_word_embedding_tutorial/wiki_ko_model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-30 05:04:58,219 : INFO : loading wv recursively from ./korean_word_embedding_tutorial/wiki_ko_model.wv.* with mmap=None
2019-07-30 05:04:58,221 : INFO : loading vectors from ./korean_word_embedding_tutorial/wiki_ko_model.wv.vectors.npy with mmap=None
2019-07-30 05:04:58,257 : INFO : setting ignored attribute vectors_norm to None
2019-07-30 05:04:58,259 : INFO : loading vocabulary recursively from ./korean_word_embedding_tutorial/wiki_ko_model.vocabulary.* with mmap=None
2019-07-30 05:04:58,264 : INFO : loading trainables re

[-1.45214461e-02  2.87395567e-01 -2.76099354e-01  2.06221461e-01
 -1.16965868e-01  2.63907909e-01  2.55381078e-01  9.38793197e-02
  2.19988320e-02 -1.82122424e-01  1.68901697e-01 -2.09141418e-01
 -1.63960516e-01  1.62283815e-02  9.56573188e-02 -2.76149929e-01
  4.76790547e-01  2.64110804e-01  2.57093031e-02  2.13133544e-02
  1.36087060e-01 -1.86460972e-01  5.70998251e-01  3.18836093e-01
 -3.35735261e-01 -1.34751666e-02 -1.34436205e-01  6.84673935e-02
 -2.13297442e-01  4.57739264e-01  4.00415719e-01  9.22798067e-02
  2.40802810e-01 -7.77181804e-01  3.41827609e-02 -7.61917140e-03
  3.22088242e-01 -1.73377708e-01  3.85171659e-02 -1.18012419e-02
 -6.35089993e-01 -8.39900002e-02 -5.06283306e-02  2.57117748e-01
 -3.63331109e-01 -1.43927500e-01 -1.38368860e-01 -4.00776386e-01
 -5.30334771e-01 -3.90796274e-01  3.82857502e-01 -1.37089416e-02
 -1.68033149e-02 -9.53079686e-02  7.75727272e-01  2.11534292e-01
 -3.31215084e-01 -3.70272756e-01 -4.58050251e-01 -1.60414442e-01
 -5.59799857e-02 -1.97982

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# 1.1 Pretrained model (부가사항)


*  **2430k lines wikipedia korea data pretrained model**
*  **본 실습모델보다 학습량이 많은 모델**




In [0]:
# 프로젝트를 하려면 이 원본데이터를 써라( 위에서 한건 작게 축소한 것.)

!mkdir ./korean_word_embedding_tutorial/pre_trained_model
!wget "https://www.dropbox.com/sh/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka?dl=0" -O pre_trained_model.zip
!unzip pre_trained_model.zip -d ./korean_word_embedding_tutorial/pre_trained_model/
!rm pre_trained_model.zip

# 깃헙에 다른나라 언어도 있다.

mkdir: cannot create directory ‘./korean_word_embedding_tutorial/pre_trained_model’: File exists
--2019-07-23 14:09:22--  https://www.dropbox.com/sh/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka [following]
--2019-07-23 14:09:22--  https://www.dropbox.com/sh/raw/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7b17222e470c13808c323dcc3e.dl.dropboxusercontent.com/zip_by_token_key?key=AlO15CZZWtv5Pqka-YruTsrffStxz8KRrs6PI-3jEf0J3y-kwzne_AdrpjgApNNA2mifhGS-lP2fGvubN0X3V4I1ZgwPwXsUtRBL--h7P2TvteyerPIuL97F0RbdmB_NGl86eIrlivHoGQC8mzUUdys3r0Ram6esB0h42Kbzw9AeajHXgfvj7B9s6FyWtY7U9UhM73r1qQVkNYSS4O

# 2. Visualization

In [0]:
# 위 결과에서 tsv파일들이 visualization 할때 필요한 자료.

!mkdir ./korean_word_embedding_tutorial/visualization

In [0]:
from collections import Counter
from gensim.models import Word2Vec
import re

In [0]:
root_path = './korean_word_embedding_tutorial/'

def main():
    
    model = Word2Vec.load(root_path + 'wiki_ko_model')
    
    
    # num_rows : model 안에 있는 단어의 갯수
    try:
      num_rows = len(model.vocab)
    except:
      model.vocab = model.wv.vocab
      num_rows = len(model.vocab)
   
    dim = model.vector_size
    
    global tensor_out_fn
    global labels_out_fn
    
    tensor_out_fn = root_path + 'visualization/wiki_ko_model_%d_%dd_tensors.tsv' % (num_rows, dim)
    labels_out_fn = root_path + 'visualization/wiki_ko_model_%d_%dd_labels.tsv' % (num_rows, dim)
    
    try:
      labels_out = open(labels_out_fn, 'w', encoding='utf-8')
    except:
      labels_out = open(labels_out_fn, 'w')
    
    
    # word(apple) language(언어 구분) count()를 labels_out에 작성
    labels_out.write('word\tlanguage\tcount\n')
    wv_list = []
    
    counter = {}
    for wd in model.vocab:
      counter[wd] = model.vocab[wd].count
    counter = Counter(counter) 
    common = counter.most_common(5000) # 카운트의 상위 5000건만 잘라서 안죽게 한다.
    words, _ = zip(*common)
    
    for wd in words:
      ww = model[wd].tolist()
      assert dim == len(ww)
      assert '\t' not in wd
      wv_list.append(ww)
      
      
      try:
        labels_out.write('%s\t%s\t%s\n'%(wd, 'ko', model.vocab[wd].count))
      except:
        labels_out.write('%s\t%s\t%s\n'%(wd, 'ko', model.vocab[wd].count)).encode('utf-8')
        
    with open(tensor_out_fn,'w') as fw:
      for i in wv_list:
        fw.write("%s\n"% (str(i).replace(', ','\t').replace('[','').replace(']','')))
        
    labels_out.close()    

In [0]:
!wget "http://bit.ly/word_label" -0 wiki_ko_model_129512_300d_labels.tsv
!wget "http://bit.ly/word_tensor" -0 wiki_ko_model_129512_300d_tensors.tsv # 자료없으면 이거 받아라.

# 이거는 여기서 train한거보다는 큰 파일이다. : 좀 다를 수 있음.

In [0]:
if __name__ =="__main__":
    main()
    
    

2019-07-30 05:20:20,211 : INFO : loading Word2Vec object from ./korean_word_embedding_tutorial/wiki_ko_model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-30 05:20:20,618 : INFO : loading wv recursively from ./korean_word_embedding_tutorial/wiki_ko_model.wv.* with mmap=None
2019-07-30 05:20:20,619 : INFO : loading vectors from ./korean_word_embedding_tutorial/wiki_ko_model.wv.vectors.npy with mmap=None
2019-07-30 05:20:20,687 : INFO : setting ignored attribute vectors_norm to None
2019-07-30 05:20:20,688 : INFO : loading vocabulary recursively from ./korean_word_embedding_tutorial/wiki_ko_model.vocabulary.* with mmap=None
2019-07-30 05:20:20,689 : INFO : loading trainables re

# [Embedding projector](http://projector.tensorflow.org/)


---


![대체 텍스트](https://miro.medium.com/max/1200/1*Fat62b1ZITOFMPXTcHNkLw.jpeg)